# convolutional variational autoencoder
- generate a bunch of new ones
- interpolate between emojis
- maxpool <> depool
- conv <> deconv

In [13]:
import os

import keras.backend as K
from keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Input, Layer as KerasLayer, Reshape
from keras.losses import mean_squared_error, binary_crossentropy, mean_absolute_error
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils.vis_utils import model_to_dot

from IPython.display import SVG
import numpy as np
import PIL
from scipy.ndimage import imread
import tensorflow as tf

In [ ]:
# import emojis

In [ ]:
EMOJIS_DIR = 'data/emojis'
N_CHANNELS = 4
EMOJI_SHAPE = (36, 36, N_CHANNELS)

In [ ]:
# try to add in emojis shaped (36, 36, 2), here, by adding empty axes

emojis = []

for slug in os.listdir(EMOJIS_DIR):
    path = os.path.join(EMOJIS_DIR, slug)
    emoji = imread(path)
    if emoji.shape == (36, 36, 4):
        emojis.append(emoji) 

emojis = np.array(emojis)

In [ ]:
train_mask = np.random.rand( len(emojis) ) < 0.8

In [ ]:
X_train = y_train = emojis[train_mask] / 255.
X_val = y_val = emojis[~train_mask] / 255.

In [ ]:
def display_emoji(emoji_arr):
    return PIL.Image.fromarray(emoji_arr)


def display_prediction(pred):
    pred = (pred * 255).astype(np.uint8)
    return display_emoji(pred)

In [ ]:
class VariationalLayer(KerasLayer):

    def __init__(self, embedding_dim: int, epsilon_std=1.):
        '''A custom "variational" Keras layer that completes the
        variational autoencoder.

        Args:
            embedding_dim : The desired number of latent dimensions in our
                embedding space.
        '''
        self.embedding_dim = embedding_dim
        self.epsilon_std = epsilon_std
        super().__init__()

    def build(self, input_shape):
        self.z_mean_weights = self.add_weight(
            shape=input_shape[-1:] + (self.embedding_dim,),
            initializer='glorot_normal',
            trainable=True,
            name='z_mean_weights'
        )
        self.z_mean_bias = self.add_weight(
            shape=(self.embedding_dim,),
            initializer='zero',
            trainable=True,
            name='z_mean_bias'
        )
        self.z_log_var_weights = self.add_weight(
            shape=input_shape[-1:] + (self.embedding_dim,),
            initializer='glorot_normal',
            trainable=True,
            name='z_log_var_weights'
        )
        self.z_log_var_bias = self.add_weight(
            shape=(self.embedding_dim,),
            initializer='zero',
            trainable=True,
            name='z_log_var_bias'
        )
        super().build(input_shape)

    def call(self, x):
        z_mean = K.dot(x, self.z_mean_weights) + self.z_mean_bias
        z_log_var = K.dot(x, self.z_log_var_weights) + self.z_log_var_bias
        epsilon = K.random_normal(
            shape=K.shape(z_log_var),
            mean=0.,
            stddev=self.epsilon_std
        )

        kl_loss_numerator = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        self.kl_loss = -0.5 * K.sum(kl_loss_numerator, axis=-1)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    def loss(self, x, x_decoded):
        base_loss = binary_crossentropy(x, x_decoded)
        base_loss = tf.reduce_sum(base_loss, axis=[-1, -2])
        return base_loss + self.kl_loss

    def compute_output_shape(self, input_shape):
        return input_shape[:1] + (self.embedding_dim,)

In [17]:
EMBEDDING_SIZE = 16
FILTER_SIZE = 64
BATCH_SIZE = 16

In [29]:
# encoder
original = Input(shape=EMOJI_SHAPE, name='original')

conv = Conv2D(filters=FILTER_SIZE, kernel_size=3, input_shape=original.shape, padding='same', activation='relu')(original)
conv = Conv2D(filters=FILTER_SIZE, kernel_size=3,padding='same', activation='relu')(conv)
conv = Conv2D(filters=FILTER_SIZE, kernel_size=3,padding='same', activation='relu')(conv)

flat = Flatten()(conv)
variational_layer = VariationalLayer(EMBEDDING_SIZE)
variational_params = variational_layer(flat)

encoder = Model([original], [variational_params], name='encoder')

# decoder
encoded = Input(shape=(EMBEDDING_SIZE,))

upsample = Dense(np.multiply.reduce(EMOJI_SHAPE), activation='relu')(encoded)
reshape = Reshape(EMOJI_SHAPE)(upsample)

deconv = Conv2DTranspose(filters=FILTER_SIZE, kernel_size=3, padding='same', activation='relu', input_shape=encoded.shape)(reshape)
deconv = Conv2DTranspose(filters=FILTER_SIZE, kernel_size=3, padding='same', activation='relu')(deconv)
deconv = Conv2DTranspose(filters=FILTER_SIZE, kernel_size=3, padding='same', activation='relu')(deconv)
reconstructed = Conv2DTranspose(filters=N_CHANNELS, kernel_size=3, padding='same', activation='sigmoid')(deconv)

decoder = Model([encoded], [reconstructed], name='decoder')

# end-to-end
encoder_decoder = Model([original], decoder(encoder([original])))

In [ ]:
# SVG(model_to_dot(encoder_decoder).create(prog='dot', format='svg'))

In [30]:
encoder_decoder.compile(optimizer=Adam(.001), loss=variational_layer.loss)

In [31]:
encoder_decoder_fit = encoder_decoder.fit(
    x=X_train[:96],
    y=y_train[:96],
    batch_size=16,
    epochs=1,
    validation_data=(X_val, y_val)
)

encoder_decoder.save('models/vae.h5')

Train on 96 samples, validate on 186 samples
Epoch 1/1
96/96 [==============================] - 8s - loss: 1910.4568 - val_loss: 882.7920


TypeError: ('Not JSON Serializable:', Dimension(None))

In [ ]:
emj = X_train[1]

In [ ]:
preds = encoder_decoder.predict(np.array([emj]))

In [ ]:
display_prediction(emj)

In [ ]:
display_prediction(preds[0])

# question-answer models for emoji responses
- when predicting, you can predict onto the generated emojis as well!
- add some dropout
- vanishing/exploding gradients
- spaCy for word2vec embeddings?

## bi-directional lstms
- "Bidirectional Long Short-Term Memory (biLSTM): Single direction LSTMs suffer a weakness
of not utilizing the contextual information from the future tokens. Bidirectional LSTM utilizes both
the previous and future context by processing the sequence on two directions, and generate two
independent sequences of LSTM output vectors. One processes the input sequence in the forward
direction, while the other processes the input in the reverse direction. The output at each time step
is the concatenation of the two output vectors from both directions, ie. ht =
−→ht k
←−ht ." (https://arxiv.org/pdf/1511.04108.pdf)

# references
- http://ben.bolte.cc/blog/2016/language.html
- https://arxiv.org/pdf/1508.01585v2.pdf
- https://arxiv.org/pdf/1511.04108.pdf
- https://explosion.ai/blog/deep-learning-formula-nlp
- https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder_deconv.py